In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pymongo
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["AttendanceSystem"]
collection = db["Users"]


base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


def faceEmba(facePIC):
    facePIC = cv2.resize(facePIC, (224, 224))
    facePIC = img_to_array(facePIC)
    facePIC = preprocess_input(facePIC)
    facePIC = np.expand_dims(facePIC, axis=0)
    embedding = base_model.predict(facePIC)[0]
    return embedding


cap = cv2.VideoCapture(0)

print("Press 's' to start capturing 25 face images.")

capEMB = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow("Register Face", frame)
    key = cv2.waitKey(1)
    if key == ord('s'):
        name = input("Enter Name: ")
        print("Capturing 25 images...")
        
        for i in range(25):
            ret, frame = cap.read()
            if not ret:
                break
            
            faceDATA = faceEmba(frame)
            capEMB.append(faceDATA)
            cv2.imshow("Capturing...", frame)
            cv2.waitKey(200)  # Wait 200ms between captures
        
        if capEMB:
            avgEMB = np.mean(capEMB, axis=0)
            userRegdata = {"name": name, "embedding": avgEMB.tolist()}
            collection.insert_one(userRegdata)
            print(f"{name} registered successfully!")
        break

cap.release()
cv2.destroyAllWindows()